In [2]:
import pandas as pd
import pickle

In [3]:
df = pd.read_csv('csv_buscas.csv',delimiter='\t')

In [4]:
df

,PMID,TITLE,ABSTRACT
0,39412306,Physicochemical and microbiological stability ...,In an effort to expedite the publication of ar...
1,39273489,Inhibitory Effects of Surface Pre-Reacted Glas...,The surface pre-reacted glass ionomer (S-PRG) ...
2,39245358,Pediatric oral extemporaneous preparations and...,This publication is the first to report curren...
3,38457115,Successful Treatment of Recurrent Clostridioid...,Fecal microbiota transplants can be administer...
4,38257156,Super Bolus-A Remedy for a High Glycemic Index...,This study aimed to compare whether a super bo...
...,...,...,...
155,2847549,Parathyroid hormone stimulates ammoniagenesis ...,To determine whether parathyroid hormone (PTH)...
156,3386134,Intracellular sodium in proximal tubules of di...,Renal hypertrophy is a common consequence of d...
157,3333125,"Giardiasis. Functional, immunological and hist...",Fourteen patients between the ages of 9 months...
158,7279470,Dental caries potential of liquid medications.,Cariogenicity of seven commonly prescribed liq...


In [5]:
#Início

In [6]:
from utils.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
)

In [7]:
EMBEDDING_MODEL = "text-embedding-3-small"

In [8]:
# load data (full dataset available at http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)
dataset_path = "csv_buscas.csv"
df = pd.read_csv(dataset_path,delimiter='\t')

n_examples = 5
df.head(n_examples)

,PMID,TITLE,ABSTRACT
0,39412306,Physicochemical and microbiological stability ...,In an effort to expedite the publication of ar...
1,39273489,Inhibitory Effects of Surface Pre-Reacted Glas...,The surface pre-reacted glass ionomer (S-PRG) ...
2,39245358,Pediatric oral extemporaneous preparations and...,This publication is the first to report curren...
3,38457115,Successful Treatment of Recurrent Clostridioid...,Fecal microbiota transplants can be administer...
4,38257156,Super Bolus-A Remedy for a High Glycemic Index...,This study aimed to compare whether a super bo...


In [18]:
# print the title, description, and label of each example
for idx, row in df.head(n_examples).iterrows():
    print("")
    print(f"PMID: {row['PMID']}")
    print(f"Title: {row['TITLE']}")
    print(f"Abstract: {row['ABSTRACT']}")


PMID: 39412306
Title: Physicochemical and microbiological stability of 40 mg/mL amiodarone hydrochloride oral suspension.
Abstract: In an effort to expedite the publication of articles, AJHP is posting manuscripts online as soon as possible after acceptance. Accepted manuscripts have been peer-reviewed and copyedited, but are posted online before technical formatting and author proofing. These manuscripts are not the final version of record and will be replaced with the final article (formatted per AJHP style and proofed by the authors) at a later time. Amiodarone hydrochloride is an antiarrhythmic drug used to treat supraventricular tachycardia. However, there are currently no commercial pediatric forms available to treat young patients. Various oral formulations were previously reported in the literature, but the concentration was lower than the doses prescribed in clinical practice (a loading dose of 500 mg/m2/day for 7-10 days followed by a maintenance dose of 250 mg/m2/day). The 

In [9]:
df_novo = df[['TITLE', 'ABSTRACT']]

In [15]:
'''
# set path to embedding cache
embedding_cache_path = "csv_buscas.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

# define a function to retrieve embeddings from the cache if present, and otherwise request via the API
def embedding_from_string(
    string: str,
    model: str = EMBEDDING_MODEL,
    embedding_cache=embedding_cache
) -> list:
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string, model)] = get_embedding(string, model)
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]

'''

In [10]:
# set path to embedding cache
embedding_cache_path = "csv_buscas.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

def embedding_from_dataset(texto: str, modelo: str = EMBEDDING_MODEL) -> list:
    """Retorna o embedding de um texto, usando cache para evitar recomputação."""
    if (texto, modelo) not in embedding_cache:
        embedding_cache[(texto, modelo)] = get_embedding(texto, modelo)
        with open(embedding_cache_path, "wb") as cache_file:
            pickle.dump(embedding_cache, cache_file)
    return embedding_cache[(texto, modelo)]

In [11]:
example_string = df["ABSTRACT"].values[0]
print(f"\nExample string: {example_string}")

# print the first 10 dimensions of the embedding
example_embedding = embedding_from_dataset(example_string)
print(example_embedding)


Example string: In an effort to expedite the publication of articles, AJHP is posting manuscripts online as soon as possible after acceptance. Accepted manuscripts have been peer-reviewed and copyedited, but are posted online before technical formatting and author proofing. These manuscripts are not the final version of record and will be replaced with the final article (formatted per AJHP style and proofed by the authors) at a later time. Amiodarone hydrochloride is an antiarrhythmic drug used to treat supraventricular tachycardia. However, there are currently no commercial pediatric forms available to treat young patients. Various oral formulations were previously reported in the literature, but the concentration was lower than the doses prescribed in clinical practice (a loading dose of 500 mg/m2/day for 7-10 days followed by a maintenance dose of 250 mg/m2/day). The objective of this study was to develop an oral liquid formulation of amiodarone hydrochloride at an optimal concentr

In [ ]:
def print_recommendations_from_strings(
    strings: list[str],
    index_of_source_string: int,
    k_nearest_neighbors: int = 1,
    model=EMBEDDING_MODEL,
) -> list[int]:
    """Print out the k nearest neighbors of a given string."""
    # get embeddings for all strings
    embeddings = [embedding_from_dataset(string, modelo=model) for string in strings]

    # get the embedding of the source string
    query_embedding = embeddings[index_of_source_string]

    # get distances between the source embedding and other embeddings (function from utils.embeddings_utils.py)
    distances = distances_from_embeddings(query_embedding, embeddings, distance_metric="cosine")
    
    # get indices of nearest neighbors (function from utils.utils.embeddings_utils.py)
    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)

    # print out source string
    query_string = strings[index_of_source_string]
    print(f"Source string: {query_string}")
    # print out its k nearest neighbors
    k_counter = 0
    for i in indices_of_nearest_neighbors:
        # skip any strings that are identical matches to the starting string
        if query_string == strings[i]:
            continue
        # stop after printing out k articles
        if k_counter >= k_nearest_neighbors:
            break
        k_counter += 1

        # print out the similar strings and their distances
        print(
            f"""
        --- Recommendation #{k_counter} (nearest neighbor {k_counter} of {k_nearest_neighbors}) ---
        String: {strings[i]}
        Distance: {distances[i]:0.3f}"""
        )

    return indices_of_nearest_neighbors

In [23]:
article_abstracts = df["ABSTRACT"].tolist()

pubmed_articles = print_recommendations_from_strings(
    strings=article_abstracts,  # let's base similarity off of the article description
    index_of_source_string=0,  # articles similar to the first one about Tony Blair
    k_nearest_neighbors=5,  # 5 most similar articles
)

Source string: In an effort to expedite the publication of articles, AJHP is posting manuscripts online as soon as possible after acceptance. Accepted manuscripts have been peer-reviewed and copyedited, but are posted online before technical formatting and author proofing. These manuscripts are not the final version of record and will be replaced with the final article (formatted per AJHP style and proofed by the authors) at a later time. Amiodarone hydrochloride is an antiarrhythmic drug used to treat supraventricular tachycardia. However, there are currently no commercial pediatric forms available to treat young patients. Various oral formulations were previously reported in the literature, but the concentration was lower than the doses prescribed in clinical practice (a loading dose of 500 mg/m2/day for 7-10 days followed by a maintenance dose of 250 mg/m2/day). The objective of this study was to develop an oral liquid formulation of amiodarone hydrochloride at an optimal concentrat

In [42]:
df

,PMID,TITLE,ABSTRACT
0,39412306,Physicochemical and microbiological stability ...,In an effort to expedite the publication of ar...
1,39273489,Inhibitory Effects of Surface Pre-Reacted Glas...,The surface pre-reacted glass ionomer (S-PRG) ...
2,39245358,Pediatric oral extemporaneous preparations and...,This publication is the first to report curren...
3,38457115,Successful Treatment of Recurrent Clostridioid...,Fecal microbiota transplants can be administer...
4,38257156,Super Bolus-A Remedy for a High Glycemic Index...,This study aimed to compare whether a super bo...
